In [1]:
from pypower.api import case9
import pandas as pd

In [2]:
# Use Case 9
ppc = case9()

# Number of buses
nb = 9

# Bus column names
col_bus = ["bus_i", "type", "Pd", "Qd", "Gs", "Bs", "area", "Vm", "Va", "baseKV", "zone", "Vmax", "Vmin"]

buses = pd.DataFrame(ppc["bus"], columns = col_bus)
buses.head(nb)


,bus_i,type,Pd,Qd,Gs,Bs,area,Vm,Va,baseKV,zone,Vmax,Vmin
0,1.0,3.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,345.0,1.0,1.1,0.9
1,2.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,345.0,1.0,1.1,0.9
2,3.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,345.0,1.0,1.1,0.9
3,4.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,345.0,1.0,1.1,0.9
4,5.0,1.0,90.0,30.0,0.0,0.0,1.0,1.0,0.0,345.0,1.0,1.1,0.9
5,6.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,345.0,1.0,1.1,0.9
6,7.0,1.0,100.0,35.0,0.0,0.0,1.0,1.0,0.0,345.0,1.0,1.1,0.9
7,8.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,345.0,1.0,1.1,0.9
8,9.0,1.0,125.0,50.0,0.0,0.0,1.0,1.0,0.0,345.0,1.0,1.1,0.9


In [3]:
# Include reactive power injections (1/4 of the active power)
ppc["gen"][:,2] = ppc["gen"][:,1] / 4

# Generator column names
col_gen = ["bus_i", "Pg", "Qg", "Qmax", "Qmin", "Vg", "mBase", "Status", "Pmax", "Pmin"]

gens = pd.DataFrame(ppc["gen"][:,0:10], columns = col_gen)
gens.head(nb)

,bus_i,Pg,Qg,Qmax,Qmin,Vg,mBase,Status,Pmax,Pmin
0,1,0,0,300,-300,1,100,1,250,10
1,2,163,40,300,-300,1,100,1,300,10
2,3,85,21,300,-300,1,100,1,270,10


In [4]:
# Branch column names
col_branch = ["fbus", "tbus", "r", "x", "b", "rateA", "rateB", "rateC"]

branches = pd.DataFrame(ppc["branch"][:,0:8], columns = col_branch)
branches.head(nb)

,fbus,tbus,r,x,b,rateA,rateB,rateC
0,1.0,4.0,0.0000,0.0576,0.000,250.0,250.0,250.0
1,4.0,5.0,0.0170,0.0920,0.158,250.0,250.0,250.0
2,5.0,6.0,0.0390,0.1700,0.358,150.0,150.0,150.0
3,3.0,6.0,0.0000,0.0586,0.000,300.0,300.0,300.0
4,6.0,7.0,0.0119,0.1008,0.209,150.0,150.0,150.0
5,7.0,8.0,0.0085,0.0720,0.149,250.0,250.0,250.0
6,8.0,2.0,0.0000,0.0625,0.000,250.0,250.0,250.0
7,8.0,9.0,0.0320,0.1610,0.306,250.0,250.0,250.0
8,9.0,4.0,0.0100,0.0850,0.176,250.0,250.0,250.0


In [ ]:
# Using CVXPY try to optimize objective function with constraints
from cvxpy import *

# Create function variables
rj = Variable(nb)
Qj = Variable(nb)
Vo = Variable(nb)
Qjplus1 = Variable(nb)
sjplus1 = Variable(nb)
qjplus1 = Variable(nb)


# Create objective function
obj = Minimize(rj * Qj**2 / Vo**2)

# Create constraints
constraints = [Qjplus1 - Qj + qjplus1 <= sjplus1,
              Qjplus1 - Qj + qjplus1 >= -sjplus1]

# Form and solve problem
prob = Problem(obj, constraints)
prob.solve()


In [ ]:
import numpy as np

# Now implement optimization via dual-ascent algorithm

# Number of iterations
itr = 1000

# Constants
Vo = 1
alpha = 0.05/Vo**2
rj = 0.33

# Initialize variables
s = np.zeros((nb,1))
s_hat = np.zeros((nb,1))

Q = np.zeros((nb,itr))
lmda1 = np.zeros((nb,itr))
lmda2 = np.zeros((nb,itr))
delta1 = np.zeros((nb, 1))
delta2 = np.zeros((nb, 1))     

for bus in range(0,nb):
    # Obtain values of consumed/generated power
    pc = buses["Pd"][bus]
    pc1 = buses["Pd"][bus+1]
    qc = buses["Qd"][bus]
    qc1 = buses["Qd"][bus+1]
    
    if bus > 2:
        pg = 0
        qg = 0
        pg1 = 0
        qg1 = 0
    elif bus == 2:
        pg = gens["Pg"][bus]
        qg = gens["Qg"][bus]
        pg1 = 0
        qg1 = 0
    else:    
        pg = gens["Pg"][bus]
        qg = gens["Qg"][bus]
        pg1 = gens["Pg"][bus+1]
        qg1 = gens["Qg"][bus+1]
    
    # Apparent power
    s[bus] = np.sqrt((pc - pg)**2 + (qc - qg)**2)
    s_hat[bus] = np.sqrt(s[bus]**2 - pg**2)
    
    if bus < (nb-1):
        s[bus+1] = np.sqrt((pc1 - pg1)**2 + (qc1 - qg1)**2)
        s_hat[bus+1] = np.sqrt(s[bus+1]**2 - pg1**2)
    
    for k in range(0,itr):
        if bus == 0:
            Q[bus, k+1] = (Vo**2 / (2*rj)) * (lmda1[bus, k] - lmda2[bus, k])
        else:
            Q[bus, k+1] = (Vo**2 / (2*rj)) * (lmda1[bus, k] - lmda1[bus-1, k] + lmda2[bus-1, k] - lmda2[bus, k])
        
        if bus <(nb-1):
            delta1[bus+1] = Q[bus+1, k+1] - Q[bus, k+1] + qc1 - s_hat[bus+1]
            delta2[bus+1] = -Q[bus+1, k+1] + Q[bus, k+1] - qc1 - s_hat[bus+1]
        
            lmda1[bus, k+1] = np.maximum([0, lmda1[bus, k] + alpha * delta1[bus+1]])
            lmda2[bus, k+1] = np.maximum([0, lmda2[bus, k] + alpha * delta2[bus+1]])
        else:
            lmda1[bus, k+1] = np.maximum([0, lmda1[bus, k]])
            lmda2[bus, k+1] = np.maximum([0, lmda2[bus, k]])
    
    

In [ ]:
# Implement Block Chain
from ethjsonrpc import EthJsonRpc, BadResponseError
connection = EthJsonRpc('127.0.0.1', 8545)
address = '0x123'

# Send transaction of Q that was solved for
Q_addr = conn.eth_accounts()[1]
tx = connection.call_with_transaction(Q_addr, address, 'setQ(int256)',[Q])